

![GMC](https://ghdisplay-live.storage.googleapis.com/upload/img_cache/file-16238-c06851b8f6cf498bb25b1381ef28afa6.jpg)

# Initialising The GMC Agent as an Issuing Authority

## This would all be done on a secure system managed by GMC IT Admin

![Secure Server](https://static.thenounproject.com/png/65152-200.png)

### Imports

In [3]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://general-medical-council-agent:3021 and an api key of adminApiKey


## GMC Public DID

![DID Document](https://www.w3.org/TR/did-core/diagrams/figure-a.1-did-and-did-document-graph.png)

The GMC uses a seed that can be set using ACAPY_WALLET_SEED environment variable. This seed is used to create a public key pair and corresponding DID. For issuers this DID must be public, meaning it has been written to indy network used by the ecosystem and has signed any necessary agreements. E.g. the Sovrin Transaction Author Agreement.

We are currently using a local von-network, seen http://localhost:9000, and have used this interface to previously author the GMC DID to the ledger.

In production, this system might use the Sovrin StagingNetwork. Where write access is limited to those DID's already on the ledger with the assigned role of Transaction Endorser. All transactions MUST be signed by a TE. This helps to limit which actors can get identifiers on the ledger and the subsequent transactions they can write.

We are assigning the GMC as a Transaction Endorser and hence root of trust for the Scottish Healthcare Ecosystem. They can then take on the role of onboarding other actors endorsing their transactions onto the ledger. For example they might sign transactions for:

* New medical school DID joining the network
* Medical school credential definition
* A specialist college joining the network
* Training programs they have regulated

Other actors are likely to trust credentials and actors who can trace the root of trust back to the GMC.

In [6]:
public_did_response = await agent_controller.wallet.get_public_did()

In [7]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

DID {'did': 'TDAbSf3Uqebg8N4XvybMbg', 'verkey': 'FHYRgHt1Z7pZBEY5yMaTHoeLMTvbQ9R6qdxhJu69xEw2', 'posture': 'public'}


## Not Needed for Local VON Network

In [6]:
# # write new DID to Sovrin Stagingnet
# import requests
# import json 

# url = 'https://selfserve.sovrin.org/nym'

# payload = {"network":"stagingnet","did": did_obj["did"],"verkey":did_obj["verkey"],"paymentaddr":""}

# # Adding empty header as parameters are being sent in payload
# headers = {}

# r = requests.post(url, data=json.dumps(payload), headers=headers)
# print(r.json())

## Not Needed for Local VON Network

## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [7]:
# taa_response = await agent_controller.ledger.get_taa()
# TAA = taa_response['result']['taa_record']
# TAA['mechanism'] = "service_agreement"
# await agent_controller.ledger.accept_taa(TAA)

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [8]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

Successfully intialised agent with Public DID :  TDAbSf3Uqebg8N4XvybMbg


## Write Trusted Medical School Schema

The GMC regulates all medical schools, ensuring they provide adequate education to medics. The GMC might want to provide medical schools with a credential, thus empowering them to digitally prove they are a trusted medical school/

In [9]:
# Define you schema name - must be unique on the ledger
schema_name = "Trusted Medical School"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["Name", "Date Issued", "Re-Validation Due"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
tms_schema_id = response["schema_id"]

## Write Credential Definition

In [12]:
# Tag and group specific credential definitions
tag = "institution"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = True

cred_def_response = await agent_controller.definitions.write_cred_def(tms_schema_id, tag, support_revocation)
tms_cred_def_id = cred_def_response["credential_definition_id"]

## Writing GMC Licence Schema

Note that the Base64DoctorImage attribute will contain a base64 image string representing an image of the doctor.

![](https://www.spine-pldd.com/public/catalogo/large/20181217_6711.jpg)

In [11]:
# Define you schema name - must be unique on the ledger
schema_name = "GMC Licence"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["GMC Number", "Responsible Officer GMC Number", "Licenced From", "Re-Validation Due", "Name", "Base64DoctorImage", "DOB"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
gmc_schema_id = response["schema_id"]

## Writing The GMC Licence Definition

The GMC Licence is the foundational credential in a healthcare professionals career. All practicing doctors in the UK must have a licence and it is subject to review every 5 years.

For this reason we have chosen to make the GMC Licence a revocable credential, such that the GMC can publish anonymous on chain updates revoking a doctors licence to practice at any time.

Clearly there would need to be some regulations and governance around this, but likely these already exist as part of the General Medical Council's processes.

Using anonymous revocation is important because it removes the requirement for a doctor to reveal a unique identifier every time they present their licence. Note if a unique identifier is required, any likely it would be a lot of the time, the interaction can request disclosure of the GMC number that identifies the doctor.

In [12]:
# Tag and group specific credential definitions
tag = "gmc"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = True

cred_def_response = await agent_controller.definitions.write_cred_def(gmc_schema_id, tag, support_revocation)
gmc_cred_def_id = cred_def_response["credential_definition_id"]

## Persist Identifiers for use throughout other business logic notebooks associated with this agent

![Database](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQvT8nx1MZBTT86OMo7ADKPW8freW47j1POHETHyCUTv-lsXGQkGCjOTZaKAvKM2L8JS3o&usqp=CAU)

Typically these would be persisted in a database within the secure server of the actor.

The schema_id and cred_def_id value pairs are required whenever issuing credentials, and also can be used to constrain acceptable proof requests. In a real application these values might be stored in environment variables or most likely in a database. For notebooks we have found it easier to store as string values in a cell and then load these values into the jupyter store so that they can be fetched across multiple notebooks.

As such you are recommended to print out each of the schema and cred def identifiers used by your agent and copy them across to your **main** business logic notebook where you should store them in a variable and save them to the jupyter store. Remember, you should only be running this notebook once so having this logic in here will not be useful.




In [13]:
print(f"tms_schema_id = '{tms_schema_id}'")
print(f"tms_cred_def_id = '{tms_cred_def_id}'")

tms_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:Trusted Medical School:0.0.1'
tms_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:13:institution'


In [14]:
dbs_schema_id = 'MKDB7pQgw1KGJZ1UtJHCkY:2:DBS Check:0.0.1'
dbs_cred_def_id = 'MKDB7pQgw1KGJZ1UtJHCkY:3:CL:216272:doctor-onboarding'

gmc_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1'
gmc_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:18:gmc'


## Terminate Controller


In [13]:
await agent_controller.terminate()